In [1]:
import os

os.environ['JAVA_HOME'] = "C:/Program Files/Java/jdk-11"
os.environ['PYSPARK_PYTHON'] = "C:/Users/usr/anaconda3/envs/pyspark_env/python.exe"
os.environ['PYSPARK_DRIVER_PYTHON'] = "C:/Users/usr/anaconda3/envs/pyspark_env/python.exe"
os.environ['HADOOP_HOME'] = "C:/hadoop-3.4.0"
os.environ['HADOOP_COMMON_LIB_NATIVE_DIR'] = "C:/hadoop-3.4.0/lib/native"
os.environ['PATH'] += os.pathsep + "C:/hadoop-3.4.0/bin"

import findspark
findspark.init()

from pyspark.sql import SparkSession

spark = SparkSession.builder.getOrCreate()

sc = spark.sparkContext

- Cree un RDD importes a partir de los datos adjuntos a esta lección como recurso. Emplee acumuladores para obtener el total de ventas realizadas y el importe total de las ventas.

In [2]:
importes = sc.textFile('./data/rdd.txt')

In [3]:
importes.collect()

['527',
 '386',
 '701',
 '240',
 '941',
 '27',
 '396',
 '56',
 '456',
 '148',
 '160',
 '229',
 '438',
 '829',
 '130',
 '386',
 '913',
 '869',
 '700',
 '93',
 '334',
 '329',
 '776',
 '882',
 '590',
 '991',
 '119',
 '721',
 '88',
 '59',
 '723',
 '118',
 '746',
 '698',
 '507',
 '585',
 '722',
 '372',
 '59',
 '969',
 '909',
 '355',
 '73',
 '601',
 '324',
 '656',
 '546',
 '318',
 '836',
 '755',
 '133',
 '373',
 '123',
 '479',
 '284',
 '801',
 '824',
 '412',
 '388',
 '609',
 '129',
 '914',
 '918',
 '510',
 '490',
 '995',
 '779',
 '813',
 '238',
 '145',
 '644',
 '506',
 '566',
 '41',
 '758',
 '828',
 '129',
 '852',
 '775',
 '283',
 '452',
 '241',
 '203',
 '753',
 '847',
 '817',
 '137',
 '163',
 '466',
 '498',
 '114',
 '31',
 '49',
 '644',
 '152',
 '36',
 '866',
 '262',
 '662',
 '811',
 '325',
 '545',
 '995',
 '634',
 '551',
 '27',
 '610',
 '464',
 '586',
 '255',
 '516',
 '12',
 '481',
 '209',
 '732',
 '913',
 '715',
 '228',
 '755',
 '293',
 '441',
 '732',
 '343',
 '995',
 '71',
 '316',
 '973'

In [4]:
contador = sc.accumulator(0)

In [5]:
def contar_importe(importe):
    global contador # variable global, es la variable de todos los ejecutores, la que tenemos creada arriba.
    contador += 1

In [6]:
importes.foreach(contar_importe)

In [27]:
# importes.foreach(lambda x: contador.add(1))

In [7]:
contador.value

10000

In [8]:
suma_total = sc.accumulator(0)

In [9]:
def suma_importe(importe):
    global suma_total
    suma_total += int(importe)


In [10]:
importes.foreach(suma_importe)

In [ ]:
# importes.foreach(lambda x: suma_total.add(int(x)))

In [11]:
suma_total.value

5042335

- Si se conoce que a cada venta hay que restarle un importe fijo igual a 10 pesos por temas de impuestos.

- ¿Cómo restaría este impuesto de cada venta utilizando una variable broadcast para acelerar el proceso?

In [12]:
diez = 10

In [13]:
br_diez = sc.broadcast(diez)

In [16]:
resta = importes.map(lambda x: int(x) - br_diez.value)

In [17]:
resta.collect()

[517,
 376,
 691,
 230,
 931,
 17,
 386,
 46,
 446,
 138,
 150,
 219,
 428,
 819,
 120,
 376,
 903,
 859,
 690,
 83,
 324,
 319,
 766,
 872,
 580,
 981,
 109,
 711,
 78,
 49,
 713,
 108,
 736,
 688,
 497,
 575,
 712,
 362,
 49,
 959,
 899,
 345,
 63,
 591,
 314,
 646,
 536,
 308,
 826,
 745,
 123,
 363,
 113,
 469,
 274,
 791,
 814,
 402,
 378,
 599,
 119,
 904,
 908,
 500,
 480,
 985,
 769,
 803,
 228,
 135,
 634,
 496,
 556,
 31,
 748,
 818,
 119,
 842,
 765,
 273,
 442,
 231,
 193,
 743,
 837,
 807,
 127,
 153,
 456,
 488,
 104,
 21,
 39,
 634,
 142,
 26,
 856,
 252,
 652,
 801,
 315,
 535,
 985,
 624,
 541,
 17,
 600,
 454,
 576,
 245,
 506,
 2,
 471,
 199,
 722,
 903,
 705,
 218,
 745,
 283,
 431,
 722,
 333,
 985,
 61,
 306,
 963,
 765,
 831,
 128,
 634,
 719,
 750,
 983,
 888,
 590,
 262,
 366,
 520,
 857,
 206,
 785,
 655,
 106,
 578,
 626,
 596,
 133,
 507,
 377,
 914,
 699,
 499,
 419,
 330,
 328,
 280,
 584,
 875,
 557,
 814,
 703,
 624,
 474,
 826,
 160,
 67,
 926,
 662,
 9

- Cree un RDD llamado ventas_sin_impuestos a partir de la propuesta del inciso a que contenga las ventas sin impuestos.

In [19]:
ventas_sin_impuestos = resta

- Destruya la variable broadcast creada luego de emplearla para crear el RDD del inciso b.

In [20]:
br_diez.destroy()

- Persista el RDD ventas_sin_impuestos en los siguientes niveles de persistencia.

- Memoria.

In [21]:
from pyspark.storagelevel import StorageLevel

In [ ]:
ventas_sin_impuestos.persist(StorageLevel.MEMORY_ONLY) # cache() es lo mimsmo.

PythonRDD[5] at collect at C:\Users\usr\AppData\Local\Temp\ipykernel_12868\548543934.py:1

- Disco solamente.

In [23]:
ventas_sin_impuestos.unpersist()

PythonRDD[5] at collect at C:\Users\usr\AppData\Local\Temp\ipykernel_12868\548543934.py:1

In [ ]:
ventas_sin_impuestos.persist(StorageLevel.DISK_ONLY)

PythonRDD[5] at collect at C:\Users\usr\AppData\Local\Temp\ipykernel_12868\548543934.py:1

- Memoria y disco.

In [25]:
ventas_sin_impuestos.unpersist()

PythonRDD[5] at collect at C:\Users\usr\AppData\Local\Temp\ipykernel_12868\548543934.py:1

In [26]:
ventas_sin_impuestos.persist(StorageLevel.MEMORY_AND_DISK)

PythonRDD[5] at collect at C:\Users\usr\AppData\Local\Temp\ipykernel_12868\548543934.py:1